In [2]:
import requests
from datetime import datetime, timedelta

ORION_URL = "http://localhost:1026"
NGSI_LD_ENDPOINT = f"{ORION_URL}/ngsi-ld/v1/entities"

print(f"Orion-LD URL: {ORION_URL}")

Orion-LD URL: http://localhost:1026


In [3]:
response = requests.get(f"{ORION_URL}/version")
print(response.json())

{'orionld version': '1.4.0', 'orion version': '1.15.0-next', 'uptime': '0 d, 6 h, 48 m, 44 s', 'git_hash': 'nogitversion', 'compile_time': 'Sat Aug 26 06:19:09 UTC 2023', 'compiled_by': 'root', 'compiled_in': '', 'release_date': 'Sat Aug 26 06:19:09 UTC 2023', 'doc': 'https://fiware-orion.readthedocs.org/en/master/'}


In [29]:
# charging session

session_start = datetime.now()
charging_start = session_start + timedelta(minutes=0.5)  

se_001 = {
#    "@context": "https://raw.githubusercontent.com/chzh63315/DigiEV/1fd2fb3374578aea96ef41b5a49fc03f144f7afd/contexts/datamodels.context-ngsi.jsonld",    
    
    # required
      "id": "urn:ngsi-ld:ChargingSession:session-512840826",
      "type": "ChargingSession",

    # required
      "refChargingPoint": {
          "type": "Relationship",
          "object": "urn:ngsi-ld:ChargingPoint:ParisSaclay30-1-P-01"
      },
      
      "sessionId": {
          "type": "Property",
          "value": "512840826"
      },

    # required
      "sessionStatus": {
          "type": "Property",
          "value": "ended"
      },
      "sessionType": {
          "type": "Property",
          "value": "AC-Level2"
      },
      
    # required
      "sessionStartTime": {
          "type": "Property",
          "value": {
              "@type": "DateTime",
              "@value": session_start.isoformat() + "Z"
          }
      },

    # required
      "chargingStartTime": {
          "type": "Property",
          "value": {
              "@type": "DateTime",
              "@value": charging_start.isoformat() + "Z"
          }
      },
      
      "sessionDuration": {
          "type": "Property",
          "value": 25.5,
          "units": "MIN",
      },
   
      "userId": {
          "type": "Property",
          "value": "RemoteApp"
      },
      "authenticationMethod": {
          "type": "Property",
          "value": "RFID"
      },
      "authenticationId": {
          "type": "Property",
          "value": "1"
      },

      "dateCreated": {
          "type": "Property",
          "value": {
              "@type": "DateTime",
              "@value": session_start.isoformat() + "Z"
          }
      },

    # required
      "lastUpdated": {
          "type": "Property",
          "value": {
              "@type": "DateTime",
              "@value": datetime.now().isoformat() + "Z"
          }
      }
  }

In [5]:
# headers = {
#     "Content-Type": "application/ld+json",
#     "Accept": "application/ld+json"
# }

headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/ld+json',
    'Link': '<https://raw.githubusercontent.com/chzh63315/DigiEV/1fd2fb3374578aea96ef41b5a49fc03f144f7afd/contexts/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"'
}

In [34]:
response = requests.post(
    NGSI_LD_ENDPOINT,
    json=se_001,
    headers=headers
)

print(f"HTTP statu: {response.status_code}")
print(f"info: {response.text}")

HTTP statu: 409
info: {"type":"https://uri.etsi.org/ngsi-ld/errors/AlreadyExists","title":"Entity already exists","detail":"urn:ngsi-ld:ChargingSession:session-512840826"}


In [7]:
query_url = f"{NGSI_LD_ENDPOINT}?type=ChargingStation"

response = requests.get(query_url, headers=headers)
print(f"HTTP status: {response.status_code}")
print(f"info: {response.json()}")

HTTP status: 200
info: []


In [38]:
query_url = f"{NGSI_LD_ENDPOINT}?type=ChargingSession"

response = requests.get(query_url, headers=headers)
print(f"HTTP status: {response.status_code}")
print(f"info: {response.json()}")

HTTP status: 200
info: [{'@context': 'https://raw.githubusercontent.com/chzh63315/DigiEV/1fd2fb3374578aea96ef41b5a49fc03f144f7afd/contexts/datamodels.context-ngsi.jsonld', 'id': 'urn:ngsi-ld:ChargingSession:session-512840826', 'type': 'ChargingSession', 'authenticationId': {'type': 'Property', 'value': '1'}, 'authenticationMethod': {'type': 'Property', 'value': 'RFID'}, 'chargingDuration': {'type': 'Property', 'value': 25, 'units': {'type': 'Property', 'value': 'MIN'}}, 'chargingStartTime': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2025-06-24T21:08:17.908435Z'}}, 'dateCreated': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2025-06-24T21:07:47.908435Z'}}, 'lastUpdated': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2025-06-24T21:07:47.909231Z'}}, 'refChargingPoint': {'type': 'Relationship', 'object': 'urn:ngsi-ld:ChargingPoint:E4C-001-CP-001'}, 'refVehicleStatus': {'type': 'Relationship', 'object': 'urn:ngsi-ld:E-VehicleStatus:vehic

In [8]:
query_url = f"{NGSI_LD_ENDPOINT}?type=ChargingSession&q=refChargingPoint==\"urn:ngsi-ld:ChargingStation:ParisSaclay30-1-P-01\""

response = requests.get(query_url, headers=headers)

print(f"HTTP status: {response.status_code}")
print(f"info: {response.json()}")

HTTP status: 200
info: []
